In [2]:
import numpy as np
from jesse import helpers, research

_, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

trading_1m.shape

(2674080, 6)

In [3]:
np.save("data/btc_1m.npy", trading_1m)

In [1]:
import numpy as np
from jesse import helpers, research

trading_1m = np.load("data/btc_1m.npy")
trading_1m.shape

(2674080, 6)

In [2]:
from custom_indicators.toolbox.dollar_bar import estimate_dollar_bar_threshold

for i in [7, 14, 14 * 6]:
    threshold = estimate_dollar_bar_threshold(trading_1m, i)
    print(f"{i}m: {threshold}")

7m: 47392922.79565829
14m: 107072898.39019096
84m: 716159708.1343918


In [3]:
from custom_indicators.config import (
    DOLLAR_BAR_THRESHOLD_7M,
    DOLLAR_BAR_THRESHOLD_14M,
    DOLLAR_BAR_THRESHOLD_84M,
)
from custom_indicators.toolbox.dollar_bar import build_dollar_bar

dollar_bar_7m = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_7M)
dollar_bar_14m = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_14M)
dollar_bar_84m = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_84M)
print(dollar_bar_7m.shape, dollar_bar_14m.shape, dollar_bar_84m.shape)

(379735, 6) (190209, 6) (32006, 6)


In [4]:
np.save("data/dollar_bar_7m.npy", dollar_bar_7m)
np.save("data/dollar_bar_84m.npy", dollar_bar_84m)

In [5]:
from custom_indicators.config import DOLLAR_BAR_THRESHOLD_14M
from custom_indicators.toolbox.dollar_bar import build_dollar_bar

candles_dollar_bar = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_14M)
candles_dollar_bar.shape

(190209, 6)

In [6]:
from custom_indicators.toolbox.labeler.zigzag_labeler import zigzag

res = zigzag(candles_dollar_bar[:, 2], 0.01)
np.unique(res, return_counts=True)

(array([-1,  1]), array([92196, 98013]))

In [7]:
label = np.column_stack((candles_dollar_bar, res))
label

array([[ 1.57784052e+12,  7.17423000e+03,  7.20210000e+03, ...,
         7.17423000e+03,  1.48385580e+04,  1.00000000e+00],
       [ 1.57786488e+12,  7.20210000e+03,  7.21576000e+03, ...,
         7.17400000e+03,  1.48677670e+04,  1.00000000e+00],
       [ 1.57788750e+12,  7.21576000e+03,  7.23532000e+03, ...,
         7.20400000e+03,  1.48482100e+04,  1.00000000e+00],
       ...,
       [ 1.73827722e+12,  1.05175100e+05,  1.05183400e+05, ...,
         1.04997700e+05,  1.03563300e+03, -1.00000000e+00],
       [ 1.73827920e+12,  1.05183400e+05,  1.04852100e+05, ...,
         1.04852000e+05,  1.11240000e+03, -1.00000000e+00],
       [ 1.73828112e+12,  1.04852000e+05,  1.04676900e+05, ...,
         1.04628300e+05,  1.11513300e+03, -1.00000000e+00]])

In [8]:
np.save("data/label_side_14m.npy", label)

In [9]:
from jesse import utils

df_dollar_bar = utils.numpy_candles_to_dataframe(candles_dollar_bar, name_date="time")
df_dollar_bar["zigzag"] = res
df_dollar_bar.head(1)

,time,open,close,high,low,volume,zigzag
2020-01-01 01:02:00,2020-01-01 01:02:00,7174.23,7202.1,7239.74,7174.23,14838.558,1


In [10]:
from lightweight_charts import Chart

chart = Chart(inner_width=1, inner_height=0.8)
chart2 = chart.create_subchart(position="bottom", width=1, height=0.2, sync=True)

chart.set(df_dollar_bar[["time", "open", "high", "low", "close", "volume"]])

line = chart2.create_line(name="zigzag", style="dotted")
line.set(df_dollar_bar[["time", "zigzag"]])

chart.show(block=True)

2025-03-07 17:41:40.717 python[20720:925309] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-07 17:41:40.717 python[20720:925309] +[IMKInputSession subclass]: chose IMKInputSession_Modern


RuntimeError: asyncio.run() cannot be called from a running event loop